In [321]:
import os,codecs
import pandas as pd
import nltk
from nltk import*
import matplotlib.pyplot as plt

In [322]:
TC = '250kSentiArt_EN.xlsx' 

In [323]:
sa = pd.read_excel(TC)

In [324]:
fn = 'XED.txt'

In [325]:
sent_mean_ang_z = []
sent_mean_fear_z = []
sent_mean_disg_z = []
sent_mean_hap_z = []
sent_mean_sad_z = []
sent_mean_surp_z = []


In [326]:
df = pd.read_csv("XED.txt",  error_bad_lines=False, sep= '\t', header= None, names = ["sent", "label_1"])

In [327]:
df.shape

(17528, 2)

In [328]:
df[["label_1", "label_2"]] = df['label_1'].str.split(",", 1, expand=True)
df[["label_2", "label_3"]] = df['label_2'].str.split(",", 1, expand=True)


In [329]:
df.head()

,sent,label_1,label_2,label_3
0,", ...",1,None,None
1,!,1,4,7
2,... And I don't think we need to discuss the T...,8,1,None
3,* So get up out of your bed,1,None,None
4,A confession that you hired [PERSON] ... and a...,1,6,None


In [330]:
row_ = df[df["sent"] == "!"].index
df.drop(row_, inplace=True)

row_1 = df[df["sent"] == ", ..."].index
df.drop(row_1, inplace=True)

In [331]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [332]:
stopwords = nltk.corpus.stopwords.words("english")

In [333]:
df["text_clean"] = df["sent"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords= stopwords))
df.head()

,sent,label_1,label_2,label_3,text_clean
2,... And I don't think we need to discuss the T...,8,1,None,dont think need discus trinity
3,* So get up out of your bed,1,None,None,get bed
4,A confession that you hired [PERSON] ... and a...,1,6,None,confession hired person responsible father murder
5,A dead man has one half - hour to raise his ro...,1,None,None,dead man one half hour raise roll outside get ...
6,A guy that's talking about he's gonna solve al...,1,None,None,guy thats talking he gonna solve problem


In [334]:
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row["text_clean"]), axis=1)
df.head()

,sent,label_1,label_2,label_3,text_clean,tokenized_sents
2,... And I don't think we need to discuss the T...,8,1,None,dont think need discus trinity,"[dont, think, need, discus, trinity]"
3,* So get up out of your bed,1,None,None,get bed,"[get, bed]"
4,A confession that you hired [PERSON] ... and a...,1,6,None,confession hired person responsible father murder,"[confession, hired, person, responsible, fathe..."
5,A dead man has one half - hour to raise his ro...,1,None,None,dead man one half hour raise roll outside get ...,"[dead, man, one, half, hour, raise, roll, outs..."
6,A guy that's talking about he's gonna solve al...,1,None,None,guy thats talking he gonna solve problem,"[guy, thats, talking, he, gon, na, solve, prob..."


In [335]:
df = df.reset_index(drop=True)

In [336]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17526 entries, 0 to 17525
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sent             17526 non-null  object
 1   label_1          17526 non-null  object
 2   label_2          3872 non-null   object
 3   label_3          848 non-null    object
 4   text_clean       17526 non-null  object
 5   tokenized_sents  17526 non-null  object
dtypes: object(6)
memory usage: 821.7+ KB


In [337]:
my_list = []
for sent in df.tokenized_sents:
    my_list.append(sent)

In [338]:
len(my_list)

17526

In [339]:
for t in my_list:
    dt = sa.query('word in @t')
    sent_mean_ang_z.append(dt.ang_z.mean())
    sent_mean_fear_z.append(dt.fear_z.mean())
    sent_mean_disg_z.append(dt.disg_z.mean())
    sent_mean_hap_z.append(dt.hap_z.mean())
    sent_mean_sad_z.append(dt.sad_z.mean())
    sent_mean_surp_z.append(dt.surp_z.mean())


In [340]:
df['ang_z'] = sent_mean_ang_z
df['fear_z'] = sent_mean_fear_z
df['disg_z'] = sent_mean_disg_z
df['hap_z'] = sent_mean_hap_z
df['sad_z'] = sent_mean_sad_z
df['surp_z'] = sent_mean_surp_z
df = round(df,3)

In [341]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17526 entries, 0 to 17525
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sent             17526 non-null  object 
 1   label_1          17526 non-null  object 
 2   label_2          3872 non-null   object 
 3   label_3          848 non-null    object 
 4   text_clean       17526 non-null  object 
 5   tokenized_sents  17526 non-null  object 
 6   ang_z            17288 non-null  float64
 7   fear_z           17288 non-null  float64
 8   disg_z           17288 non-null  float64
 9   hap_z            17288 non-null  float64
 10  sad_z            17288 non-null  float64
 11  surp_z           17288 non-null  float64
dtypes: float64(6), object(6)
memory usage: 1.6+ MB


In [342]:
print(df.ang_z.isna().sum())      
print(df.fear_z.isna().sum())   
print(df.disg_z.isna().sum()) 
print(df.hap_z.isna().sum())
print(df.sad_z.isnull().sum())
print(df.surp_z.isnull().sum())

238
238
238
238
238
238


In [343]:
df_238 = df.loc[pd.isna(df["disg_z"]), :]
df_238.head(30)

,sent,label_1,label_2,label_3,text_clean,tokenized_sents,ang_z,fear_z,disg_z,hap_z,sad_z,surp_z
29,After them ! After them !,1,2,None,,[],NaN,NaN,NaN,NaN,NaN,NaN
113,And now this ?,1,None,None,,[],NaN,NaN,NaN,NaN,NaN,NaN
117,And so will you .,1,2,None,,[],NaN,NaN,NaN,NaN,NaN,NaN
200,Argh ! Kaili !,1,None,None,argh kaili,"[argh, kaili]",NaN,NaN,NaN,NaN,NaN,NaN
285,Bullshit !,1,3,None,bullshit,[bullshit],NaN,NaN,NaN,NaN,NaN,NaN
287,Bullshit .,1,7,None,bullshit,[bullshit],NaN,NaN,NaN,NaN,NaN,NaN
322,But you can not do this !,1,None,None,,[],NaN,NaN,NaN,NaN,NaN,NaN
417,Coochie ...,1,None,None,coochie,[coochie],NaN,NaN,NaN,NaN,NaN,NaN
421,Copycat !,1,None,None,copycat,[copycat],NaN,NaN,NaN,NaN,NaN,NaN
449,Dickhead .,1,None,None,dickhead,[dickhead],NaN,NaN,NaN,NaN,NaN,NaN


In [344]:
df_238.to_csv('df_238.csv')